In [8]:
from py2neo import Graph
from pandas import DataFrame

graph = Graph(password="neo4j")
k = DataFrame(graph.data("MATCH (a:Person) RETURN a.name, a.born"))

In [9]:
#為了方便後面做Index用，建立a-z,A-Z的26*2個英文字母，加上5個特殊符號，共57個符號
import string
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

#建立函式方便尋找all_letters的Index像是"a"=10000000000....
def letterToIndex(letter):
    return all_letters.find(letter)

# 將一個字母轉成Tensor顯示
def letterToTensor(letter):
    tensor = np.zeros((1, n_letters)) #建立一個1*57的Tensor
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# 建立one-hot letter vectors
def lineToTensor(line):
    tensor = np.zeros((len(line), 1, n_letters))
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

#print(letterToTensor('b')) #將內容轉為字母 #ex. a,b,c..
#print(lineToTensor('Jones')) #5為字母個數、1為維度、57為總字母

In [10]:
a = lineToTensor(k['a.name'][0][0])[0][0]
for number in range(1,len(k['a.name'])):
    ltt = lineToTensor(k['a.name'][number][0])
    b = ltt[0][0]
    a = np.vstack((a, b))

In [11]:
import tensorflow as tf
import numpy as np

# Mini-batch: instance 0,instance 1,instance 2,instance 3
#X0_batch = np.array([[0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0]]) # t = 0
#X1_batch = np.array([[0, 1, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]) # t = 1
X0_batch = a[0:133]
X1_batch = a[133:266]

n_inputs = 57
n_neurons = 5
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons],dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})
    print(Y0_val)
    print(Y1_val)

[[-0.95848954 -0.1928485   0.34492564  0.09297504  0.67311645]
 [ 0.18878719  0.5443703  -0.82238    -0.50262773 -0.41096047]
 [-0.27648923  0.03865335  0.10430115 -0.8553978  -0.2453807 ]
 [-0.27648923  0.03865335  0.10430115 -0.8553978  -0.2453807 ]
 [ 0.8532283   0.7811952  -0.31147745 -0.661653   -0.5020549 ]
 [ 0.20302097  0.99836195  0.4171776   0.65339667  0.80316037]
 [ 0.30294132  0.81045365  0.7047397   0.73220825  0.1329982 ]
 [ 0.30294132  0.81045365  0.7047397   0.73220825  0.1329982 ]
 [ 0.6673807  -0.26451483  0.78146154  0.2940622  -0.9376573 ]
 [-0.95848954 -0.1928485   0.34492564  0.09297504  0.67311645]
 [ 0.30294132  0.81045365  0.7047397   0.73220825  0.1329982 ]
 [ 0.8532283   0.7811952  -0.31147745 -0.661653   -0.5020549 ]
 [ 0.8532283   0.7811952  -0.31147745 -0.661653   -0.5020549 ]
 [-0.95848954 -0.1928485   0.34492564  0.09297504  0.67311645]
 [-0.61698824  0.7324035   0.27637714  0.4694205   0.9783592 ]
 [ 0.18878719  0.5443703  -0.82238    -0.50262773 -0.41